In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import acquire
import prepare

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [2]:
iris = acquire.get_iris_data(cached=True)
prepare.prep_iris(iris)

,species,sepal_length,sepal_width,petal_length,petal_width,species_versicolor,species_virginica
0,setosa,5.1,3.5,1.4,0.2,0,0
1,setosa,4.9,3.0,1.4,0.2,0,0
2,setosa,4.7,3.2,1.3,0.2,0,0
3,setosa,4.6,3.1,1.5,0.2,0,0
4,setosa,5.0,3.6,1.4,0.2,0,0
...,...,...,...,...,...,...,...
145,virginica,6.7,3.0,5.2,2.3,0,1
146,virginica,6.3,2.5,5.0,1.9,0,1
147,virginica,6.5,3.0,5.2,2.0,0,1
148,virginica,6.2,3.4,5.4,2.3,0,1


In [ ]:
iris = acquire.get_iris_data(cached=True)
iris.describe()